In [1]:
import azureml.core

print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

You are currently using version 1.0.18 of the Azure ML SDK

##### Create Azure ML Workspace, if it doesn't exist

In [3]:
from azureml.core import Workspace

subscription_id = "4dd7cdaa-1664-46be-b521-237c98ccf3f6"

ws = Workspace.create(name="ML-Service-Workspace",
                      subscription_id=subscription_id,    
                      resource_group="ML-Service-RG",
                      create_resource_group=True,
                      location="eastus",
                      exist_ok=True)

##### Write workspace configuration file to local FS

In [5]:
ws.write_config()

Wrote the config file config.json to: /databricks/driver/aml_config/config.json

##### Copy workspace configuration file to DBFS, to persist it

In [7]:
%sh cp /databricks/driver/aml_config/config.json /dbfs/tmp

##### Copy serialized model file from DBFS to local FS

In [9]:
#NOTE: Need to copy model from DBFS to local FS, as Azure ML service deployment always gets the model from the current working dir
import os

base_path = "/tmp/rf_model/"
model_name = "rf_model"

model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(base_path, model_local, True)

local_path = os.getcwd() + "/" + model_name

copy model from dbfs to local

In [10]:
%sh ls -la /databricks/driver/rf_model/rf_model.mllib

total 16
drwxr-xr-x 4 root root 4096 Mar 20 00:47 .
drwxr-xr-x 3 root root 4096 Mar 20 00:47 ..
drwxr-xr-x 2 root root 4096 Mar 20 00:47 metadata
drwxr-xr-x 5 root root 4096 Mar 20 00:47 stages

##### Register the model on Azure ML service

In [12]:
#Register the model
from azureml.core.model import Model

mymodel = Model.register(model_path = model_name + "/rf_model.mllib", # local path where the model file was copied to
                         model_name = model_name, # this is the name to be used to register the model on Azure ML service
                         description = "MLlib Randomforest Model",
                         workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model rf_model
rf_model MLlib Randomforest Model 15

##### Create a model scoring script and save it to local FS

In [14]:
score_sparkml = """

import json
import pickle
import pyspark
from azureml.core.model import Model
from pyspark.ml import PipelineModel
from pyspark.ml.feature import IndexToString

def init():
    # One-time initialization of PySpark and predictive model

    global trainedModel
    global spark

    spark = pyspark.sql.SparkSession.builder.appName("MLlib Randomforest Model Operationalization").getOrCreate()
    model_name = "{model_name}" #interpolated
    model_path = Model.get_model_path(model_name)
    trainedModel = PipelineModel.load(model_path)
    
def run(input_json):
    if isinstance(trainedModel, Exception):
        return json.dumps({{"trainedModel":str(trainedModel)}})
      
    try:
        sc = spark.sparkContext
        input_list = [json.loads(input_json)]
        input_rdd = sc.parallelize(input_list)
        input_df = spark.read.json(input_rdd)
    
        # Compute prediction
        prediction = trainedModel.transform(input_df)
        idx_to_string = IndexToString(inputCol="prediction", outputCol="classe_predicted", labels=trainedModel.stages[-3].labels)
        prediction = idx_to_string.transform(prediction)
        
        #result = prediction.first().prediction
        predictions = prediction.collect()

        #Get each scored result
        preds = [str(x["classe_predicted"]) for x in predictions]
        result = ",".join(preds)
        # you can return any data type as long as it is JSON-serializable
        return json.dumps({{"result":result}})        
    except Exception as e:
        result = str(e)
        return json.dumps({{"error":result}})
    
""".format(model_name=model_name)

exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

##### Create a deployment configuration file, optionally listing any dependent packages

In [16]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas']) #showing how to add libs as an example - not needed for this model.

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

##### Create deployment configuration

In [18]:
#Deployment configuration
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Databricks Azure ML ACI'}, 
    description = 'ACI to serve ADB Randomforest model.')

##### Create an image to host the model scoring script and deploy the web service

In [20]:
#This will take 10-15 minutes to finish

service_name = "aciadbws"
runtime = "spark-py" 
driver_file = "score_sparkml.py"
my_conda_file = "mydeployenv.yml"

# image creation
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                    runtime = runtime,
                                                    conda_file = my_conda_file)

# Webservice creation
myservice = Webservice.deploy_from_model(workspace=ws,
                                         name=service_name,
                                         deployment_config = myaci_config,
                                         models = [mymodel],
                                         image_config = myimage_config)

myservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image aciadbws:12, operation "Succeeded"
Creating service
Running......................................................
SucceededACI service creation operation finished, operation "Succeeded"

In [21]:
print(myservice.state)

Healthy

In [22]:
print(myservice.get_logs())

2019-03-20 01:35:01,050 CRIT Supervisor running as root (no user in config file)
2019-03-20 01:35:01,052 INFO supervisord started with pid 1
2019-03-20 01:35:02,054 INFO spawned: 'rsyslog' with pid 9
2019-03-20 01:35:02,056 INFO spawned: 'program_exit' with pid 10
2019-03-20 01:35:02,057 INFO spawned: 'nginx' with pid 11
2019-03-20 01:35:02,058 INFO spawned: 'iot' with pid 12
2019-03-20 01:35:02,060 INFO spawned: 'gunicorn' with pid 13
2019-03-20 01:35:02,080 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)
2019-03-20 01:35:02,145 INFO exited: iot (exit status 1; expected)
{"timestamp": "2019-03-20T01:35:02.488453Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-2086907bdabd4da2bade8b2c57ad64b6-8efc4719bdd604d136c157", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-03-20T01:35:02.489105Z", "message": "Listening at: http://127.0.0.1:9090 (13)", "host": "wk-caas-2086907bdabd4da2bade8b2c57ad64b6-8efc4719bdd604d136c157", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Listening at: %s (%s)", "stack_info": null}
{"timestamp": "2019-03-20T01:35:02.489364Z", "message": "Using worker: sync", "host": "wk-caas-2086907bdabd4da2bade8b2c57ad64b6-8efc4719bdd604d136c157", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Using worker: %s", "stack_info": null}
{"timestamp": "2019-03-20T01:35:02.489933Z", "message": "worker timeout is set to 300", "host": "wk-caas-2086907bdabd4da2bade8b2c57ad64b6-8efc4719bdd604d136c157", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "stack_info": null}
{"timestamp": "2019-03-20T01:35:02.490786Z", "message": "Booting worker with pid: 35", "host": "wk-caas-2086907bdabd4da2bade8b2c57ad64b6-8efc4719bdd604d136c157", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Booting worker with pid: %s", "stack_info": null}
2019-03-20 01:35:03,492 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2019-03-20 01:35:03,493 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/home/mmlspark/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.ml.spark#mmlspark_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-263675ef-0995-449d-bbf9-67147be79fb6;1.0
	confs: [default]
	found com.microsoft.ml.spark#mmlspark_2.11;0.15 in spark-list
	found io.spray#spray-json_2.11;1.3.2 in central
	found com.microsoft.cntk#cntk;2.4 in central
	found org.openpnp#opencv;3.2.0-1 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents#httpclient;4.5.6 in central
	found org.apache.httpcomponents#httpcore;4.4.10 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.10 in central
	found com.microsoft.ml.lightgbm#lightgbmlib;2.1.250 in central
:: resolution report :: resolve 477ms :: artifacts dl 53ms
	:: modules in use:
	com.jcraft#jsch;0.1.54 from central in [default]
	com.microsoft.cntk#cntk;2.4 from central 